# Detecting AI-Generated Images Using a Hybrid ResNet-SE Attention Model


### Mounting the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Importing the Libraries

In [5]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from lime import lime_image
import shap
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import os
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, f1_score

### Loading the Dataset

In [7]:
dataset_path = "D:/IndependentStudy/AIGeneratedImageIdentification/Dataset"
img_height = 224
img_width  = 224
batch_size = 32

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_path, "train"), 
    validation_split = 0.2, subset = "training", seed = 123, image_size = (img_height, img_width), batch_size = batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_path, "train"), 
    validation_split = 0.2, subset = "validation", seed = 123, image_size = (img_height, img_width), batch_size = batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_path, "test"), 
    image_size = (img_height, img_width), batch_size = batch_size, shuffle = False)


class_names = train_ds.class_names
print("Class names:", class_names)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Found 100000 files belonging to 2 classes.
Using 20000 files for validation.
Found 20000 files belonging to 2 classes.
Class names: ['FAKE', 'REAL']


### Pre-processing & Normalize the Data

In [9]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x,   y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x,  y: (normalization_layer(x), y))


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size = AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size = AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size = AUTOTUNE)

### Model Architecture

####  Squeeze - and - Excitation (SE) Attention Block

In [10]:
def se_block(input_tensor, reduction_ratio = 16):
    channel_axis = -1
    filters = input_tensor.shape[channel_axis]

    se = layers.GlobalAveragePooling2D()(input_tensor)
    se = layers.Dense(filters // reduction_ratio, activation = 'relu')(se)
    se = layers.Dense(filters, activation = 'sigmoid')(se)
    se = layers.Reshape((1, 1, filters))(se)
    return layers.multiply([input_tensor, se])

#### ResNet-SE Model

In [11]:
def build_resnet_se_model(input_shape = (224, 224, 3)):
    base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
    base_model.trainable = True

    inputs = layers.Input(shape = input_shape)
    x = base_model(inputs, training = True)
    x = se_block(x)  
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation = 'relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation = 'sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

* The model integrates a ResNet50 backbone with a Squeeze-and-Excitation (SE) block to enhance feature channel importance using attention.

* Useing transfer learning with ImageNet weights and supports fine-tuning for better performance on custom binary classification tasks.

* A combination of global average pooling, dense layers, and dropout enables efficient feature reduction and regularization.

### Compile Model

In [ ]:
model = build_resnet_se_model()

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
              metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

* The model has ~24.3 million trainable parameters, primarily from the ResNet50 backbone, enabling strong feature extraction.

* An SE block integrated after ResNet50 to apply channel-wise attention, enhancing important features.

* The architecture ends with dense layers and dropout, suited for binary classification tasks with added regularization.


### Model Training 

In [ ]:
history = model.fit(train_ds, validation_data = val_ds, epochs = 10)

### Model Evaluation 

In [ ]:
def plot_history(history):
    acc     = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    prec     = history.history['precision']
    val_prec = history.history['val_precision']
    
    recall     = history.history['recall']
    val_recall = history.history['val_recall']
    
    loss     = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize = (16, 10))

    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label = 'Train Accuracy')
    plt.plot(epochs_range, val_acc, label = 'Val Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, prec, label = 'Train Precision')
    plt.plot(epochs_range, val_prec, label = 'Val Precision')
    plt.legend()
    plt.title('Precision')

    plt.subplot(2, 2, 3)
    plt.plot(epochs_range, recall, label = 'Train Recall')
    plt.plot(epochs_range, val_recall, label = 'Val Recall')
    plt.legend()
    plt.title('Recall')

    plt.subplot(2, 2, 4)
    plt.plot(epochs_range, loss, label = 'Train Loss')
    plt.plot(epochs_range, val_loss, label = 'Val Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

plot_history(history)

### Evaluate on Train & Test

In [ ]:
train_loss, train_acc, train_prec, train_recall = model.evaluate(train_ds)
test_loss, test_acc, test_prec, test_recall     = model.evaluate(test_ds)

y_pred_probs = model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

y_true = np.concatenate([y for x, y in test_ds], axis = 0)

f1 = f1_score(y_true, y_pred)

print(f"\nTrain Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_recall:.4f}")
print(f"Test  Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_recall:.4f}, F1-score: {f1:.4f}")

### Evaluation Matrix - Confusion Matrix & Classification Report

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names = class_names))


#### ROC Curve and AUC

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize = (8, 6))
plt.plot(fpr, tpr, label = f"AUC = {roc_auc:.4f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc = "lower right")
plt.grid()
plt.show()

* **High AUC Score (0.9862):** The ROC curve shows excellent discrimination between classes, indicating that the model is highly effective at distinguishing real vs fake inputs.

* **Strong Test Performance:** The test accuracy is 96.12%, with precision of 97.04%, recall of 88.94%, and F1-score of 92.82%, confirming the model’s robustness and balanced generalization on unseen data.

* **Efficient Use of Epochs:** Only 10 epochs were used for training due to time and compute constraints, even when using a GPU. Despite the limited training duration, the model achieved high accuracy and strong generalization. 
                
                Effective transfer learning from ImageNet-pretrained ResNet50.
                Squeeze-and-Excitation (SE) blocks helped quickly boost feature focus.



* **Confusion Matrix Insights:** 
            
            FAKE samples: 9729 correctly classified, 271 false positives
            REAL samples: 8894 correctly classified, 1106 false negatives

            This indicates the model is slightly more conservative in detecting real instances.

* **Train vs Test Consistency:** With train accuracy at 98.97% and test accuracy at 96.12%, there’s minimal overfitting, showing that fine-tuning with the SE-ResNet model generalizes well.

* **Balanced Class Performance:** Both classes (FAKE and REAL) show high precision and recall, with macro and weighted averages of 0.93, indicating balanced and reliable classification.



#### Conclusion:

The ResNet50-SE model achieves excellent performance on binary classification of real vs fake images, with high accuracy and a strong AUC. Minor improvements can be made by addressing the false negatives using techniques like data augmentation or class-balanced loss.